In [1]:
from smtutil import hiveutil, oracle_to_hive, spark_demographic, common, weblog_util, location_util
import pandas as pd
import pandas.io.sql as psql
from pyspark.sql.functions import lag, lead, first, last, udf, lit, col, array_sort, collect_list, expr, row_number, lit,concat
from pyspark.sql.window import Window
import pyspark.sql.functions as f
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, StructType, StructField, DoubleType, BooleanType, LongType
from pyspark.sql import functions as func
from datetime import datetime as dt
import datetime
import json
from datetime import date, timedelta
import os
import numpy as np
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)

Connect to Oracle Successfully
Connect to Oracle Successfully


## Data Source
#### Data Usage Agg
1. res_house_moving 
2. res_hkbn
3. res_netvigator
4. res_village_property_agent
#### Call Record 
1. sdf_hkbn
2. sdf_netvigator
#### 5GBB CHRUN List
1. sdf_FBB

## Aggragation
#### Union all Web Access
1. res_merged=res_house_moving.union(res_house_renovating).union(res_hkbn).union(res_netvigator).cache()
2. res_pivot=res_merged.groupBy('date_id','SUBR_NUM').pivot('APPS').agg(f.sum('TOTAL_BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),f.countDistinct('date_id').alias('TOTAL_DAYS')).na.fill(0).withColumnRenamed('date_id','DATE_ID')         

#### Union and pivot hotline call
1. res_2n_call_Log=sdf_2n_call_log.groupby('DATE_ID','SUBR_NUM').pivot('network_operator').agg(f.sum('CALL_DUR').alias('CALL_DUR')).na.fill(0)

#### Outer Join Agg Web SDF and Agg Hotline SDF, left join live active sim
1. res=res_2n_call_Log.join(res_pivot,['DATE_ID','SUBR_NUM'],'outer').na.fill(0).join(sdf_postpaid_mas_n1,['SUBR_NUM'],'left').where('CUST_NUM is not null')

#### Join the sdf result with Chrun List
1. sdf_FBB

In [2]:
# spark.stop()
# conn.close()
# sc.stop()

In [3]:
sc, spark = hiveutil.get_sc_spark()

In [4]:
from weblog_util.common import getCleansedDomainList, getAggValue, getDomainSummary # , getAggValue_ver1

In [5]:
conn=oracle_to_hive.connect_to_Oracle()

Connect to Oracle Successfully


In [6]:
# function to get dates
def get_date_vars(file_export_date, bw_date_st=3, duration=2, bw_date_st_village=8):
    
    fed = dt.strptime(file_export_date, '%Y%m%d')
    
    # get 3 days if Monday
    if fed.weekday() == 0:
        bw_date_st = 4
        duration = 3
    
    d_call_start=dt.strftime(fed-timedelta(days=bw_date_st), '%Y-%m-%d')
    d_call_end=dt.strftime(fed-timedelta(days=bw_date_st-duration+1), '%Y-%m-%d')
    val_agg_period_start=int(d_call_start.replace('-', ''))
    val_agg_period_end=int(d_call_end.replace('-', ''))

    val_agg_period_start_village=int(dt.strftime(fed-timedelta(days=bw_date_st_village), '%Y%m%d'))

    return d_call_start, d_call_end, val_agg_period_start, val_agg_period_end, val_agg_period_start_village

## Parameters

In [7]:
file_export_date='20230117'
# get today
# file_export_date=dt.strftime(dt.now(), '%Y%m%d')
output_path = f'/app/sna_bdmrdev02/data_out/to_Cognos_De_SMS/De_5G_BB_{file_export_date}.csv'

# get date vars
d_call_start, d_call_end, val_agg_period_start, val_agg_period_end, val_agg_period_start_village=get_date_vars(file_export_date)

In [8]:
print(d_call_start, d_call_end, val_agg_period_start, val_agg_period_end, val_agg_period_start_village)
print(output_path)

2023-01-14 2023-01-15 20230114 20230115 20230109
/app/sna_bdmrdev02/data_out/to_Cognos_De_SMS/De_5G_BB_20230117.csv


### Postpaid Master

sdf_postpaid_mas=spark.sql("""
select
    MONTH_ID,
    CUST_NUM,
    SUBR_NUM,
    AGE,
    GENDER,
    CURRENT_SHK_PLAN_GRP,
    CURRENT_SHK_PLAN_SUBGRP
from 
    prd_biz_summ_vw.vw_shk_postpaid_master_summ
where NOMINATION_FLG = 'N'
and CURRENT_SHK_PLAN_SUBGRP not in ('Excluded','MVNOs (Local)')
and MONTH_ID = 202205
and PREPOST_FLAG = 'PO'
and REVENUE_SUBR_FLAG = 'Y'
and ACTIVE_SUBR_FLAG = 'Y'
""")

### PostPaid Mas N1

In [9]:
df_postpaid_mas_n1 = pd.read_sql_query("""
select
                VW_PREPD_POSTPAID_SUBR_N1.CUST_NUM,
                VW_PREPD_POSTPAID_SUBR_N1.SUBR_NUM,
                VW_PREPD_POSTPAID_SUBR_N1.D_HKID_N

from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1
left join PRD_BIZ_SUMM_VW.VW_RATE_PLAN_REF
on VW_PREPD_POSTPAID_SUBR_N1.Rate_Plan_Cd = VW_RATE_PLAN_REF.Rate_Plan_Cd

left join PRD_BIZ_SUMM_VW.VW_PREPD_DEMO_CARD
on VW_PREPD_POSTPAID_SUBR_N1.Subr_Num = VW_PREPD_DEMO_CARD.Msisdn

left join PRD_BIZ_SUMM_VW.VW_SUBR_NOMINATION_HIST 
on VW_PREPD_POSTPAID_SUBR_N1.Cust_Num = VW_SUBR_NOMINATION_HIST.CUST_NUM
and VW_PREPD_POSTPAID_SUBR_N1.subr_num = VW_SUBR_NOMINATION_HIST.subr_num

left join PRD_BIZ_SUMM_VW.VW_SUBR_DATA_ENTITLE
on VW_PREPD_POSTPAID_SUBR_N1.Cust_Num = VW_SUBR_DATA_ENTITLE.CUST_NUM
and  VW_PREPD_POSTPAID_SUBR_N1.subr_num = VW_SUBR_DATA_ENTITLE.subr_num

left join PRD_BIZ_SUMM_VW.VW_SHK_RATE_PLAN_GRP_REF
on VW_SUBR_DATA_ENTITLE.FREE_DATA_ENTITLE = VW_SHK_RATE_PLAN_GRP_REF.FREE_DATA_ENTITLE

WHERE VW_PREPD_POSTPAID_SUBR_N1.D_Active_Subr_Flg = 'Y'
                AND VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type = 'PO'
                AND CASE 
                                WHEN (
                                                                VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type <> 'PO'
                                                                AND VW_PREPD_DEMO_CARD.Msisdn <> ' '
                                                                )
                                                THEN 'N'
                                WHEN (
                                                                (coalesce(VW_RATE_PLAN_REF.Rate_Plan_Grp, ' ')) IN (
                                                                                'Add-on Numbers'
                                                                                ,'Non Revenue Mobile Plan'
                                                                                ,'Non Revenue BB Line'
                                                                                ,'DHL Special Project'
                                                                                )
                                                                )
                                                THEN 'N'
                                WHEN (
                                                                (coalesce(VW_PREPD_POSTPAID_SUBR_N1.Cust_Type_Cd, ' ')) IN (
                                                                                'SMC'
                                                                                ,'LPUC1'
                                                                                ,'HKBU1'
                                                                                ,'RWSU1'
                                                                                ,'HKBU2'
                                                                                ,'GCU1'
                                                                                )
                                                                )
                                                THEN 'N'
                                ELSE 'Y'
                                END = 'Y'
                AND VW_PREPD_POSTPAID_SUBR_N1.NC_FLG = 'N'
                AND VW_SHK_RATE_PLAN_GRP_REF.SHK_PLAN_SUBGRP NOT IN (
                                'Excluded'
                                ,'MVNOs (Local)'
                                ,'MVNOs (Roaming)'
                                )
                AND VW_SUBR_NOMINATION_HIST.NOMINATION_FLG = 'N'
""",con=conn)

In [10]:
sdf_postpaid_mas_n1=spark.createDataFrame(df_postpaid_mas_n1)

In [11]:
sdf_postpaid_mas_n1.limit(2).toPandas()

CUST_NUM  SUBR_NUM    D_HKID_N
0  09721480  66500278  Z932838(7)
1  09721877  96176998  M299881(0)

In [12]:
df_hkid_5gbb=pd.read_sql("""
select distinct
    D_HKID_N
    , 'Y' as Cur_5GBB_by_HKID
from PRD_biz_summ_vw.vw_prepd_postpaid_subr_n1
where SUBR_NUM like '19943%'
OR SUBR_NUM like '19944%'
AND SUBR_STAT_CD = 'OK'
""", con=conn)

In [13]:
sdf_hkid_5gbb=spark.createDataFrame(df_hkid_5gbb)

In [14]:
df_hkid_5gbb.head(2)

D_HKID_N CUR_5GBB_BY_HKID
0  E222138(5)                Y
1  Y373631(6)                Y

In [15]:
df_5gbb = pd.read_sql("""
SELECT   A.CUST_NUM, A.SUBR_NUM as BB_subr_num, B.SUBR_NUM as mobile_subr_num, 'Y' as Currently_5GBB_user
FROM
(
SELECT * FROM PRD_biz_summ_vw.vw_prepd_postpaid_subr_n1  WHERE SUBR_NUM LIKE '19943%' OR SUBR_NUM LIKE '19944%' AND SUBR_STAT_CD='OK'  
) a
LEFT OUTER JOIN
(
SELECT * FROM PRD_biz_summ_vw.vw_prepd_postpaid_subr_n1  WHERE SUBR_NUM not  LIKE '19943%' OR SUBR_NUM not LIKE '19944%' AND SUBR_STAT_CD='OK'  
) b
on a.cust_num = b.cust_num
AND A.SUBR_NUM <> B.SUBR_NUM
where length(B.SUBR_NUM) = 8
""", con=conn)

In [16]:
sdf_5gbb=spark.createDataFrame(df_5gbb)

In [17]:
sdf_5gbb.limit(2).show()

+--------+-----------+---------------+-------------------+
|CUST_NUM|BB_SUBR_NUM|MOBILE_SUBR_NUM|CURRENTLY_5GBB_USER|
+--------+-----------+---------------+-------------------+
|07697813| 1994308744|       96667255|                  Y|
|07981553| 1994416005|       93181995|                  Y|
+--------+-----------+---------------+-------------------+



#### House Moving

In [18]:
house_moving_list=[
'WWW.MEKOTRANSPORT.COM',
'WWW.PPSMOVING.COM.HK',
'WWW.CANAAN.HK',
'WWW.CHUNGPONG.COM.HK',
'WWW.PRUDENTIALMOVERS.COM',
'YANYAN.HK',
'WWW.HOITONG.COM',
'WWW.RINGOMOVINGCOMPANY.COM',
'YANYAN.HK',
'WWW.JUMBOMOVERS.COM.HK',
'WWW.EAMOVERS.COM.HK',
'WWW.A-ONE.HK',
'TUNGWAHMOVE.COM',
'WWW.PPSMOVING.COM.HK',
'WWW.27048498.COM',
'WWW.MOVINGCOMBINE.COM',
'PIANOMOVINGKING.COM',
'WWW.PIANOMOVINGKING.COM',
'WWW.GOINGMOVINGHK.COM',
'WWW.JPSTYLEMOVING.COM',
'WWW.MOVE-MOVING.COM',
'WWW.KWONGLEEMOVING.HK',
'WWW.TOPWAYMOVING.COM',
'WWW.PRICERIGHT.COM.HK',
'PRICERIGHT.COM.HK',
'WWW.28506666.COM.HK'
]

In [19]:
# house_moving_age_value=getAggValue(house_moving_list)
res_house_moving=getDomainSummary(agg_value_table=getAggValue(house_moving_list),
                                  agg_freq='daily',
                                  agg_period_start=val_agg_period_start,
                                  agg_period_end=val_agg_period_end,
                                  app_name='HOUSE_MOVING',
                                  filter_table=sdf_postpaid_mas_n1)

In [20]:
res_house_moving.limit(2).show()

+--------+--------+----------------+---------+------------+
|SUBR_NUM| date_id|TOTAL_BYTE_COUNT|TOTAL_HIT|        APPS|
+--------+--------+----------------+---------+------------+
|98416639|20230114|          251704|        1|HOUSE_MOVING|
|91835034|20230114|            5806|        1|HOUSE_MOVING|
+--------+--------+----------------+---------+------------+



#### HKBN

In [21]:
hkbn_list = ['HKBN.NET']

In [22]:
res_hkbn=getDomainSummary(agg_value_table=getAggValue(hkbn_list),
                                  agg_freq='daily',
                                  agg_period_start=val_agg_period_start,
                                  agg_period_end=val_agg_period_end,
                                  app_name='HKBN',
                                  filter_table=sdf_postpaid_mas_n1)

In [23]:
res_hkbn.limit(2).show()

+--------+--------+----------------+---------+----+
|SUBR_NUM| date_id|TOTAL_BYTE_COUNT|TOTAL_HIT|APPS|
+--------+--------+----------------+---------+----+
|55007107|20230115|           17920|        2|HKBN|
|60185262|20230114|         1199107|       10|HKBN|
+--------+--------+----------------+---------+----+



#### Netvigator

In [24]:
netvigator_list=['NETVIGATOR.COM']

In [25]:
# house_moving_age_value=getAggValue(house_moving_list)
res_netvigator=getDomainSummary(agg_value_table=getAggValue(netvigator_list),
                                  agg_freq='daily',
                                  agg_period_start=val_agg_period_start,
                                  agg_period_end=val_agg_period_end,
                                  app_name='NETVIGATOR',
                                  filter_table=sdf_postpaid_mas_n1)

In [26]:
res_netvigator.limit(2).show()

+--------+--------+----------------+---------+----------+
|SUBR_NUM| date_id|TOTAL_BYTE_COUNT|TOTAL_HIT|      APPS|
+--------+--------+----------------+---------+----------+
|61221068|20230115|        32040124|        8|NETVIGATOR|
|63490065|20230115|           36444|        4|NETVIGATOR|
+--------+--------+----------------+---------+----------+



### Village Property Agent 

In [27]:
#### Village Property Agent
village_property_agent_list=[
'http://www.kahing.com.hk/',
'https://cnp.hk/',
'http://pinnacleproperty.com.hk/',
'http://www.easyhouse.hk/',
'https://www.home-expert.hk/',
'https://www.imperialprop.com.hk/'
]

### Use function getDomainSummary to get customer weblog in villaga property agent 
1. APPS col value should be 'VILLAGE'

In [28]:
res_village_property_agent=getDomainSummary(agg_value_table=getAggValue(village_property_agent_list),
                                  agg_freq='daily',
                                  agg_period_start=val_agg_period_start_village,
                                  agg_period_end=val_agg_period_end,
                                  app_name='VILLAGE',
                                  filter_table=sdf_postpaid_mas_n1)

In [29]:
### Expected Result
res_village_property_agent.limit(10).show()
# SUBR_NUM|date_id|TOTAL_BYTE_COUNT|TOTAL_HIT|APPS

+--------+--------+----------------+---------+-------+
|SUBR_NUM| date_id|TOTAL_BYTE_COUNT|TOTAL_HIT|   APPS|
+--------+--------+----------------+---------+-------+
|56993128|20230114|           46511|        3|VILLAGE|
|64995525|20230109|           70061|        2|VILLAGE|
|97411364|20230113|          259463|        1|VILLAGE|
|55971777|20230110|           61503|        1|VILLAGE|
|98698239|20230110|           15889|        2|VILLAGE|
|67675515|20230111|         2799654|        2|VILLAGE|
|96691536|20230110|            3908|        4|VILLAGE|
|97033274|20230115|         3426414|        2|VILLAGE|
|98053145|20230115|          518524|        2|VILLAGE|
|98053145|20230114|         2110357|        3|VILLAGE|
+--------+--------+----------------+---------+-------+



### House Renovating Companies & Platform and Inspection Companines

In [30]:
#### Home Renovating Companies
house_renovating_list=[
'https://www.gahomeservice.com/21321asdsa',
'https://sanmarinodesign.com/dasd',
'https://www.carol-construction.com/1232',
'https://www.fungstyle.com/asd',
'https://mystylehouse.com.hk/zxc',
'https://www.eleganthouse.net/asdas',
#### Home Renovating Matching Platforms
'https://codeco.hk'
'https://echouse.com.hk/',
'https://www.spaceplan.com.hk/',
'https://renovate.qanvast.com/',
'https://hkdecoman.com/',
'http://easyfull.hk',
'http://www.renobro.com.hk/',
#### House Inspection Website
'https://www.hksurveyors.com/',
'https://www.silverconeng.com.hk/',
'https://hk-ppi.com/',
'https://www.legendflyhk.com/'
]

house_renovating_list=getCleansedDomainList(house_renovating_list)

In [31]:
res_house_renovating=getDomainSummary(agg_value_table=getAggValue(house_renovating_list),
                                  agg_freq='daily',
                                  agg_period_start=val_agg_period_start,
                                  agg_period_end=val_agg_period_end,
                                  app_name='HOUSE_RENOVATING_COMPANYS',
                                  filter_table=sdf_postpaid_mas_n1)

#### HKBN & NETVIGATOR Hotline

In [32]:
# Get Sunday
import pandas as pd

def allsundays(year):
    return pd.date_range(start=str(year), end=str(year+1), 
                         freq='W-SUN').strftime('%Y-%m-%d').tolist()

sundays_list=allsundays(int(d_call_start[:4]))

In [33]:
netvigator_hotline='28880089'
hkbn_hotline='34902020'
# hkbn_hotline=['34913888']

In [34]:
df_hkbn=pd.read_sql(f"""
with t2 as (Select A_NUM, CUST_NUM, SUBR_NUM, CALL_DUR, CALL_START_DATE, case when CALL_START_DATE between trunc(CALL_START_DATE + 1,'iw')-1 and trunc(CALL_START_DATE + 1,'iw')+5 then trunc(CALL_START_DATE + 1,'iw')-1 end as date_id
from prd_biz_summ_vw.vw_end_user_msc_call_dtl where call_start_date between DATE '{d_call_start}' and DATE '{d_call_end}' and A_NUM = '{hkbn_hotline}')
Select CUST_NUM,SUBR_NUM, date_id, sum(CALL_DUR) as CALL_DUR from t2 group by CUST_NUM, SUBR_NUM,date_id
""",con=conn)

In [35]:
df_netvigator=pd.read_sql(f"""
with t2 as (Select A_NUM, CUST_NUM, SUBR_NUM, CALL_DUR, CALL_START_DATE, case when CALL_START_DATE between trunc(CALL_START_DATE + 1,'iw')-1 and trunc(CALL_START_DATE + 1,'iw')+5 then trunc(CALL_START_DATE + 1,'iw')-1 end as date_id
from prd_biz_summ_vw.vw_end_user_msc_call_dtl where call_start_date between DATE '{d_call_start}' and DATE '{d_call_end}' and A_NUM = '{netvigator_hotline}')
Select CUST_NUM,SUBR_NUM, date_id, sum(CALL_DUR) as CALL_DUR from t2 group by CUST_NUM, SUBR_NUM, date_id
""",con=conn)

In [36]:
df_hkbn.head()

CUST_NUM  SUBR_NUM    DATE_ID  CALL_DUR
0  09112622  61129598 2023-01-15       757
1  06612048  92578660 2023-01-08       984
2  12478816  90833811 2023-01-08       770
3  12445768  92021171 2023-01-15       148
4  11997933  66230754 2023-01-08        78

In [37]:
df_netvigator.head(5)

CUST_NUM  SUBR_NUM    DATE_ID  CALL_DUR
0  06961214  95464864 2023-01-08       108
1  08056381  98748248 2023-01-08       512
2  12446276  66079198 2023-01-15       296

In [38]:
df_hkbn['DATE_ID']=df_hkbn['DATE_ID'].apply(lambda x: x.strftime('%Y%m%d'))
df_netvigator['DATE_ID']=df_netvigator['DATE_ID'].apply(lambda x: x.strftime('%Y%m%d'))

In [39]:
# schema_hkbn_netvi = StructType([ \
#     StructField("CUST_NUM",StringType(),True), \
#     StructField("SUBR_NUM",StringType(),True), \
#     StructField("DATE_ID",StringType(),True), \
#     StructField("CALL_DUR", LongType(), True) \
#   ])

# try:
#     sdf_hkbn=spark.createDataFrame(df_hkbn)
# except ValueError:
#     print('[Info] HKBN empty')
#     sdf_hkbn=spark.createDataFrame(df_hkbn, schema=schema_hkbn_netvi)
    
# try:
#     sdf_netvigator=spark.createDataFrame(df_netvigator)
# except ValueError:
#     print('[Info] Netvigator empty')
#     sdf_netvigator=spark.createDataFrame(df_netvigator, schema=schema_hkbn_netvi)

In [40]:
sdf_hkbn=spark.createDataFrame(df_hkbn)
sdf_netvigator=spark.createDataFrame(df_netvigator)

In [41]:
sdf_hkbn=sdf_hkbn.withColumn('network_operator',lit('HKBN'))
sdf_netvigator=sdf_netvigator.withColumn('network_operator',lit('NETVIGATOR'))

#### FBB Churn List

In [42]:
df_fbb = pd.read_sql_query("""
SELECT
    DISTINCT
    VW_PREPD_POSTPAID_CUST.Cust_Num AS CUST_NUM
    ,VW_PREPD_POSTPAID_SUBR_N1.Subr_Num AS SUBR_NUM
    ,VW_HKID_BR.Hkid_Br AS Hkid_Br
    ,VW_PREPD_POSTPAID_SUBR_N1.Subr_Sw_On_Date AS Subscriber_Switch_On_Date
    ,VW_PREPD_POSTPAID_SUBR_N1.ACCS_MAX_SU_EFFECT_DATE AS Subscriber_Switch_Off_Date
FROM (
    SELECT CUST_NUM AS Cust_Num
        ,SUBR_NUM AS Subr_Num
        ,SUBR_STAT_CD AS Subr_Stat_Cd
        ,SUBR_SW_ON_DATE AS Subr_Sw_On_Date
        ,D_ACTIVE_SUBR_FLG AS D_Active_Subr_Flg
        ,RATE_PLAN_CD AS Rate_Plan_Cd
        ,ACCS_MAX_SU_EFFECT_DATE AS Accs_Max_SU_Effect_Date
    FROM PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1
    ) VW_PREPD_POSTPAID_SUBR_N1
LEFT OUTER JOIN (

    SELECT
        RATE_PLAN_CD AS Rate_Plan_Cd
    FROM PRD_BIZ_SUMM_VW.VW_PREPD_POSTPD_RATE_PLAN_REF
    ) VW_PREPD_POSTPD_RATE_PLAN_REF ON (VW_PREPD_POSTPAID_SUBR_N1.Rate_Plan_Cd = VW_PREPD_POSTPD_RATE_PLAN_REF.Rate_Plan_Cd)
LEFT OUTER JOIN (
    SELECT STAT_CD AS Stat_Cd
    FROM PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_STAT_REF
    ) VW_PREPD_POSTPAID_STAT_REF ON (VW_PREPD_POSTPAID_SUBR_N1.Subr_Stat_Cd = VW_PREPD_POSTPAID_STAT_REF.Stat_Cd)
LEFT OUTER JOIN (
    SELECT CUST_NUM AS Cust_Num
    FROM PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_CUST
    ) VW_PREPD_POSTPAID_CUST ON (VW_PREPD_POSTPAID_CUST.Cust_Num = VW_PREPD_POSTPAID_SUBR_N1.Cust_Num)
LEFT OUTER JOIN (
    SELECT CUST_NUM AS Cust_Num
        ,HKID_BR AS Hkid_Br
    FROM PRD_BIZ_SUMM_VW.VW_CUST_MISC_INFO
    ) VW_CUST_MISC_INFO ON (VW_PREPD_POSTPAID_CUST.Cust_Num = VW_CUST_MISC_INFO.Cust_Num)
LEFT OUTER JOIN (
    SELECT HKID_BR_PREFIX AS Hkid_Br_Prefix
        ,HKID_BR AS Hkid_Br
    FROM PRD_BIZ_SUMM_VW.VW_HKID_BR
    ) VW_HKID_BR ON (VW_CUST_MISC_INFO.Hkid_Br = VW_HKID_BR.Hkid_Br)
LEFT OUTER JOIN (
    SELECT HKID_BR_PREFIX AS Hkid_Br_Prefix
    FROM PRD_BIZ_SUMM_VW.VW_HKID_BR_PREFIX
    ) VW_HKID_BR_PREFIX ON (VW_HKID_BR.Hkid_Br_Prefix = VW_HKID_BR_PREFIX.Hkid_Br_Prefix)
LEFT OUTER JOIN (
    SELECT HKID_BR_PREFIX AS Hkid_Br_Prefix
        ,D_HKID_ACTIVE_FL_CNT AS D_Hkid_Active_FL_Cnt
        ,D_HKID_ACTIVE_MOBILE_CNT_S
    FROM PRD_BIZ_SUMM_VW.VW_HKID_MISC_INFO
    ) VW_HKID_MISC_INFO ON (VW_HKID_BR_PREFIX.Hkid_Br_Prefix = VW_HKID_MISC_INFO.Hkid_Br_Prefix)
LEFT OUTER JOIN (
    SELECT CONSENT_FLG AS Consent_Flg
        ,CUST_NUM AS Cust_Num
    FROM PRD_BIZ_SUMM_VW.VW_DM_CONSENT_INFO_HKID
    ) VW_DM_CONSENT_INFO_HKID ON (VW_PREPD_POSTPAID_CUST.Cust_Num = VW_DM_CONSENT_INFO_HKID.Cust_Num)
LEFT OUTER JOIN (
    SELECT SUBR_NUM AS Subr_Num
        ,CUST_NUM AS Cust_Num
        ,TELESALES AS Telesales
    FROM PRD_BIZ_SUMM_VW.VW_ALL_PROFILING_NEW
    ) VW_ALL_PROFILING_NEW ON (
        (VW_ALL_PROFILING_NEW.Cust_Num = VW_PREPD_POSTPAID_SUBR_N1.Cust_Num)
        AND (VW_ALL_PROFILING_NEW.Subr_Num = VW_PREPD_POSTPAID_SUBR_N1.Subr_Num)
        )
LEFT OUTER JOIN (
    SELECT SUBR_NUM AS Subr_Num
        ,CUST_NUM AS Cust_Num
        ,REJECT_ALL_COMM_FLG AS Reject_All_Comm_Flg
    FROM PRD_BIZ_SUMM_VW.VW_ALL_PROFILING_N2
    ) VW_ALL_PROFILING_N2 ON (
        (VW_ALL_PROFILING_N2.Cust_Num = VW_PREPD_POSTPAID_SUBR_N1.Cust_Num)
        AND (VW_ALL_PROFILING_N2.Subr_Num = VW_PREPD_POSTPAID_SUBR_N1.Subr_Num)
        )
LEFT OUTER JOIN (
    SELECT CONSENT_FLG AS Consent_Flg
        ,CUST_NUM AS Cust_Num
        ,SUBR_NUM AS Subr_Num
    FROM PRD_BIZ_SUMM_VW.VW_DM_CONSENT_INFO_SUBR_NUM
    ) VW_DM_CONSENT_INFO_SUBR_NUM ON (
        (VW_DM_CONSENT_INFO_SUBR_NUM.Cust_Num = VW_PREPD_POSTPAID_SUBR_N1.Cust_Num)
        AND (VW_DM_CONSENT_INFO_SUBR_NUM.Subr_Num = VW_PREPD_POSTPAID_SUBR_N1.Subr_Num)
        )
LEFT OUTER JOIN (
    SELECT CUST_NUM AS Cust_Num
        ,SUBR_NUM AS Subr_Num
        ,LM_LD_CD AS Lm_Ld_Cd
        ,LM_LD_EXPIRED_DATE AS Lm_Ld_Expired_Date
        ,LM2_LD_CD AS Lm2_Ld_Cd
        ,LM2_LD_EXPIRED_DATE AS Lm2_Ld_Expired_Date
    FROM PRD_BIZ_SUMM_VW.VW_LATEST_SUBR_LD_PART1
    ) VW_LATEST_SUBR_LD_PART1 ON (
        (VW_LATEST_SUBR_LD_PART1.Cust_Num = VW_PREPD_POSTPAID_SUBR_N1.Cust_Num)
        AND (VW_LATEST_SUBR_LD_PART1.Subr_Num = VW_PREPD_POSTPAID_SUBR_N1.Subr_Num)
        )
LEFT OUTER JOIN (
    SELECT SUBR_NUM AS Subr_Num
        ,CUST_NUM AS Cust_Num
        ,ESTATE_TYPE AS Estate_Type
    FROM PRD_BIZ_SUMM_VW.VW_FBB_INSTALL_INFO
    ) VW_FBB_INSTALL_INFO ON (
        (VW_FBB_INSTALL_INFO.Cust_Num = VW_PREPD_POSTPAID_SUBR_N1.Cust_Num)
        AND (VW_FBB_INSTALL_INFO.Subr_Num = VW_PREPD_POSTPAID_SUBR_N1.Subr_Num)
        )
WHERE (
        (
            (VW_PREPD_POSTPAID_SUBR_N1.Subr_Num LIKE '15%')
            OR (VW_PREPD_POSTPAID_SUBR_N1.Subr_Num LIKE '18%')
            )
        OR (VW_PREPD_POSTPAID_SUBR_N1.Subr_Num LIKE '16%')
        )
    AND (VW_PREPD_POSTPAID_SUBR_N1.Subr_Sw_On_Date < VW_PREPD_POSTPAID_SUBR_N1.Accs_Max_SU_Effect_Date)
    AND (NOT (VW_PREPD_POSTPD_RATE_PLAN_REF.Rate_Plan_Cd IN ('FB00F')))
    and VW_HKID_BR.Hkid_Br is not null
    and VW_HKID_BR.Hkid_Br <> ' '
""",con=conn)
df_fbb['SUBSCRIBER_SWITCH_ON_DATE'] = df_fbb['SUBSCRIBER_SWITCH_ON_DATE'].apply(lambda x: x.strftime('%Y-%m-%d')).astype(str)
df_fbb['SUBSCRIBER_SWITCH_OFF_DATE'] = df_fbb['SUBSCRIBER_SWITCH_OFF_DATE'].apply(lambda x: x.strftime('%Y-%m-%d')).astype(str)
sdf_FBB=spark.createDataFrame(df_fbb)
# Find FBB Churned Customer for 18 Months and 30 Months
sdf_FBB=sdf_FBB.where("SUBSCRIBER_SWITCH_OFF_DATE between DATE '2020-02-01' and DATE '2022-08-01' ")

#### User Language

In [43]:
df_lang=pd.read_sql_query("""
with t as (
select
                VW_PREPD_POSTPAID_SUBR_N1.CUST_NUM,
                VW_PREPD_POSTPAID_SUBR_N1.SUBR_NUM

from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1
left join PRD_BIZ_SUMM_VW.VW_RATE_PLAN_REF
on VW_PREPD_POSTPAID_SUBR_N1.Rate_Plan_Cd = VW_RATE_PLAN_REF.Rate_Plan_Cd

left join PRD_BIZ_SUMM_VW.VW_PREPD_DEMO_CARD
on VW_PREPD_POSTPAID_SUBR_N1.Subr_Num = VW_PREPD_DEMO_CARD.Msisdn

left join PRD_BIZ_SUMM_VW.VW_SUBR_NOMINATION_HIST 
on VW_PREPD_POSTPAID_SUBR_N1.Cust_Num = VW_SUBR_NOMINATION_HIST.CUST_NUM
and VW_PREPD_POSTPAID_SUBR_N1.subr_num = VW_SUBR_NOMINATION_HIST.subr_num

left join PRD_BIZ_SUMM_VW.VW_SUBR_DATA_ENTITLE
on VW_PREPD_POSTPAID_SUBR_N1.Cust_Num = VW_SUBR_DATA_ENTITLE.CUST_NUM
and  VW_PREPD_POSTPAID_SUBR_N1.subr_num = VW_SUBR_DATA_ENTITLE.subr_num

left join PRD_BIZ_SUMM_VW.VW_SHK_RATE_PLAN_GRP_REF
on VW_SUBR_DATA_ENTITLE.FREE_DATA_ENTITLE = VW_SHK_RATE_PLAN_GRP_REF.FREE_DATA_ENTITLE

WHERE VW_PREPD_POSTPAID_SUBR_N1.D_Active_Subr_Flg = 'Y'
                AND VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type = 'PO'
                AND CASE 
                                WHEN (
                                                                VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type <> 'PO'
                                                                AND VW_PREPD_DEMO_CARD.Msisdn <> ' '
                                                                )
                                                THEN 'N'
                                WHEN (
                                                                (coalesce(VW_RATE_PLAN_REF.Rate_Plan_Grp, ' ')) IN (
                                                                                'Add-on Numbers'
                                                                                ,'Non Revenue Mobile Plan'
                                                                                ,'Non Revenue BB Line'
                                                                                ,'DHL Special Project'
                                                                                )
                                                                )
                                                THEN 'N'
                                WHEN (
                                                                (coalesce(VW_PREPD_POSTPAID_SUBR_N1.Cust_Type_Cd, ' ')) IN (
                                                                                'SMC'
                                                                                ,'LPUC1'
                                                                                ,'HKBU1'
                                                                                ,'RWSU1'
                                                                                ,'HKBU2'
                                                                                ,'GCU1'
                                                                                )
                                                                )
                                                THEN 'N'
                                ELSE 'Y'
                                END = 'Y'
                AND VW_PREPD_POSTPAID_SUBR_N1.NC_FLG = 'N'
                AND VW_SHK_RATE_PLAN_GRP_REF.SHK_PLAN_SUBGRP NOT IN (
                                'Excluded'
                                ,'MVNOs (Local)'
                                ,'MVNOs (Roaming)'
                                )
                AND VW_SUBR_NOMINATION_HIST.NOMINATION_FLG = 'N'
)

    select distinct
         t.cust_num
        ,t.subr_num
        ,a.D_LANG_PREFERENCE as LANG
    from PRD_BIZ_SUMM_VW.VW_PREPD_POST_COMM_LANG a
    join t
    on t.CUST_NUM = a.CUST_NUM and t.SUBR_NUM = a.SUBR_NUM
""",con=conn)

In [44]:
sdf_lang=spark.createDataFrame(df_lang)

In [45]:
sdf_lang.limit(2).show()

+--------+--------+-------+
|CUST_NUM|SUBR_NUM|   LANG|
+--------+--------+-------+
|09721480|66500278|Chinese|
|10718421|96896464|Chinese|
+--------+--------+-------+



In [46]:
# from dateutil.relativedelta import relativedelta
# datetime.datetime.strptime('2022-08-01','%Y-%m-%d')+relativedelta(months=-18)
# datetime.datetime.strptime('2022-08-01','%Y-%m-%d')+relativedelta(months=-30)

#### Pivot & Merge

In [47]:
# res_merged=res_house_moving.union(res_house_renovating).union(res_hkbn).union(res_netvigator).cache()

In [48]:
res_merged=res_house_moving.union(res_house_renovating).union(res_hkbn).union(res_netvigator).union(res_village_property_agent).cache()

In [49]:
res_merged.limit(2).show()

+--------+--------+----------------+---------+------------+
|SUBR_NUM| date_id|TOTAL_BYTE_COUNT|TOTAL_HIT|        APPS|
+--------+--------+----------------+---------+------------+
|98416639|20230114|          251704|        1|HOUSE_MOVING|
|91835034|20230114|            5806|        1|HOUSE_MOVING|
+--------+--------+----------------+---------+------------+



In [50]:
# temp=[res_house_moving, res_house_renovating, res_hkbn, res_netvigator, res_village_property_agent]
# for x in temp:
#     print(x.count())

In [51]:
res_pivot=res_merged.groupBy('date_id','SUBR_NUM')\
                    .pivot('APPS').agg(f.sum('TOTAL_BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),\
                                       f.countDistinct('date_id').alias('TOTAL_DAYS')\
                                      ).na.fill(0).withColumnRenamed('date_id','DATE_ID')

In [52]:
res_pivot.limit(2).toPandas()

DATE_ID  SUBR_NUM  HKBN_TOTAL_BYTE_COUNT  HKBN_TOTAL_DAYS  \
0  20230114  61235707                      0                0   
1  20230115  91383038                      0                0   

   HOUSE_MOVING_TOTAL_BYTE_COUNT  HOUSE_MOVING_TOTAL_DAYS  \
0                              0                        0   
1                              0                        0   

   HOUSE_RENOVATING_COMPANYS_TOTAL_BYTE_COUNT  \
0                                           0   
1                                           0   

   HOUSE_RENOVATING_COMPANYS_TOTAL_DAYS  NETVIGATOR_TOTAL_BYTE_COUNT  \
0                                     0                      3007593   
1                                     0                      2465336   

   NETVIGATOR_TOTAL_DAYS  VILLAGE_TOTAL_BYTE_COUNT  VILLAGE_TOTAL_DAYS  
0                      1                         0                   0  
1                      1                         0                   0

In [53]:
# res_pivot=res_merged.groupBy('date_id','SUBR_NUM').pivot('APPS').agg(f.sum('TOTAL_BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),\
#                                                                  f.sum('MAX_DAYS').alias('TOTAL_DAYS')).na.fill(0).withColumnRenamed('date_id','DATE_ID')

#### Union HKBN Hotline & Netvigator Hotline

In [54]:
sdf_2n_call_log=sdf_hkbn.union(sdf_netvigator)

In [55]:
sdf_2n_call_log.limit(2).show()

+--------+--------+--------+--------+----------------+
|CUST_NUM|SUBR_NUM| DATE_ID|CALL_DUR|network_operator|
+--------+--------+--------+--------+----------------+
|09112622|61129598|20230115|     757|            HKBN|
|06612048|92578660|20230108|     984|            HKBN|
+--------+--------+--------+--------+----------------+



In [56]:
res_2n_call_Log=sdf_2n_call_log.groupby('DATE_ID','SUBR_NUM').pivot('network_operator').agg(f.sum('CALL_DUR').alias('CALL_DUR')).na.fill(0)

In [57]:
res_2n_call_Log.limit(2).toPandas()

DATE_ID  SUBR_NUM  HKBN  NETVIGATOR
0  20230108  98774018   423           0
1  20230108  91504921    64           0

In [58]:
res_pivot.limit(2).toPandas()

DATE_ID  SUBR_NUM  HKBN_TOTAL_BYTE_COUNT  HKBN_TOTAL_DAYS  \
0  20230114  61235707                      0                0   
1  20230115  91383038                      0                0   

   HOUSE_MOVING_TOTAL_BYTE_COUNT  HOUSE_MOVING_TOTAL_DAYS  \
0                              0                        0   
1                              0                        0   

   HOUSE_RENOVATING_COMPANYS_TOTAL_BYTE_COUNT  \
0                                           0   
1                                           0   

   HOUSE_RENOVATING_COMPANYS_TOTAL_DAYS  NETVIGATOR_TOTAL_BYTE_COUNT  \
0                                     0                      3007593   
1                                     0                      2465336   

   NETVIGATOR_TOTAL_DAYS  VILLAGE_TOTAL_BYTE_COUNT  VILLAGE_TOTAL_DAYS  
0                      1                         0                   0  
1                      1                         0                   0

In [59]:
sdf_5gbb=sdf_5gbb.withColumnRenamed('MOBILE_SUBR_NUM', 'SUBR_NUM')

In [60]:
sdf_5gbb.limit(2).show()

+--------+-----------+--------+-------------------+
|CUST_NUM|BB_SUBR_NUM|SUBR_NUM|CURRENTLY_5GBB_USER|
+--------+-----------+--------+-------------------+
|07697813| 1994308744|96667255|                  Y|
|07981553| 1994416005|93181995|                  Y|
+--------+-----------+--------+-------------------+



In [61]:
res=res_2n_call_Log.join(res_pivot,['DATE_ID','SUBR_NUM'],'outer').na.fill(0)\
                   .join(sdf_postpaid_mas_n1,['SUBR_NUM'],'left').where('CUST_NUM is not null')\
                   .join(sdf_5gbb,['CUST_NUM', 'SUBR_NUM'],'left').na.fill('N')\
                   .join(sdf_hkid_5gbb,['D_HKID_N'],'left').na.fill('N')\
                   .join(sdf_lang,['CUST_NUM', 'SUBR_NUM'], 'left')

In [62]:
# sdf_5gbb.limit(2).show()

In [63]:
res.limit(2).toPandas()

CUST_NUM  SUBR_NUM    D_HKID_N   DATE_ID  HKBN  NETVIGATOR  \
0  00063978  90168434  G542303(0)  20230114     0           0   
1  00412957  61166173  E983297(5)  20230114     0           0   

   HKBN_TOTAL_BYTE_COUNT  HKBN_TOTAL_DAYS  HOUSE_MOVING_TOTAL_BYTE_COUNT  \
0                      0                0                              0   
1                1211219                1                              0   

   HOUSE_MOVING_TOTAL_DAYS  HOUSE_RENOVATING_COMPANYS_TOTAL_BYTE_COUNT  \
0                        0                                           0   
1                        0                                           0   

   HOUSE_RENOVATING_COMPANYS_TOTAL_DAYS  NETVIGATOR_TOTAL_BYTE_COUNT  \
0                                     0                      1006116   
1                                     0                            0   

   NETVIGATOR_TOTAL_DAYS  VILLAGE_TOTAL_BYTE_COUNT  VILLAGE_TOTAL_DAYS  \
0                      1                         0                   0   
1                      0                         0                   0   

  BB_SUBR_NUM CURRENTLY_5GBB_USER CUR_5GBB_BY_HKID     LANG  
0           N                   N                N  English  
1           N                   N                N  Chinese

In [64]:
res_FBB=sdf_FBB.select(col('CUST_NUM'),col('SUBSCRIBER_SWITCH_OFF_DATE')).withColumn('FBB_CHURN_USER',lit('Y'))

In [65]:
res_FBB.limit(2).toPandas()

CUST_NUM SUBSCRIBER_SWITCH_OFF_DATE FBB_CHURN_USER
0  12058376                 2020-11-04              Y
1  12105356                 2022-06-05              Y

In [66]:
res=res.join(res_FBB,['CUST_NUM'],'outer')

In [67]:
res=res.fillna('N','FBB_CHURN_USER')

In [68]:
res.count()

45365

In [69]:
res.limit(2).toPandas()

CUST_NUM  SUBR_NUM    D_HKID_N   DATE_ID  HKBN  NETVIGATOR  \
0  00248730  94551290  G411696(7)  20230114     0           0   
1  00564213  91717174  P257057(6)  20230114     0           0   

   HKBN_TOTAL_BYTE_COUNT  HKBN_TOTAL_DAYS  HOUSE_MOVING_TOTAL_BYTE_COUNT  \
0                      0                0                              0   
1                      0                0                              0   

   HOUSE_MOVING_TOTAL_DAYS  HOUSE_RENOVATING_COMPANYS_TOTAL_BYTE_COUNT  \
0                        0                                           0   
1                        0                                           0   

   HOUSE_RENOVATING_COMPANYS_TOTAL_DAYS  NETVIGATOR_TOTAL_BYTE_COUNT  \
0                                     0                       420335   
1                                     0                     29320496   

   NETVIGATOR_TOTAL_DAYS  VILLAGE_TOTAL_BYTE_COUNT  VILLAGE_TOTAL_DAYS  \
0                      1                         0                   0   
1                      1                         0                   0   

  BB_SUBR_NUM CURRENTLY_5GBB_USER CUR_5GBB_BY_HKID     LANG  \
0  1994354395                   Y                Y  Chinese   
1           N                   N                N  English   

  SUBSCRIBER_SWITCH_OFF_DATE FBB_CHURN_USER  
0                       None              N  
1                       None              N

In [70]:
res.printSchema()

root
 |-- CUST_NUM: string (nullable = true)
 |-- SUBR_NUM: string (nullable = true)
 |-- D_HKID_N: string (nullable = true)
 |-- DATE_ID: string (nullable = true)
 |-- HKBN: long (nullable = true)
 |-- NETVIGATOR: long (nullable = true)
 |-- HKBN_TOTAL_BYTE_COUNT: long (nullable = true)
 |-- HKBN_TOTAL_DAYS: long (nullable = true)
 |-- HOUSE_MOVING_TOTAL_BYTE_COUNT: long (nullable = true)
 |-- HOUSE_MOVING_TOTAL_DAYS: long (nullable = true)
 |-- HOUSE_RENOVATING_COMPANYS_TOTAL_BYTE_COUNT: long (nullable = true)
 |-- HOUSE_RENOVATING_COMPANYS_TOTAL_DAYS: long (nullable = true)
 |-- NETVIGATOR_TOTAL_BYTE_COUNT: long (nullable = true)
 |-- NETVIGATOR_TOTAL_DAYS: long (nullable = true)
 |-- VILLAGE_TOTAL_BYTE_COUNT: long (nullable = true)
 |-- VILLAGE_TOTAL_DAYS: long (nullable = true)
 |-- BB_SUBR_NUM: string (nullable = true)
 |-- CURRENTLY_5GBB_USER: string (nullable = true)
 |-- CUR_5GBB_BY_HKID: string (nullable = true)
 |-- LANG: string (nullable = true)
 |-- SUBSCRIBER_SWITCH_OFF_D

In [71]:
res.filter(col('CURRENTLY_5GBB_USER') == 'N')\
    .filter(col('Cur_5GBB_by_HKID') != 'Y')\
    .select(\
           col('DATE_ID').alias('DATE')\
           ,'CUST_NUM', 'SUBR_NUM'\
           ,col('HKBN').alias('HKBN_RET_HOTLINE_IN_MINS')\
           ,col('NETVIGATOR').alias('NETVIGATOR_RET_HOTLINE_IN_MINS')\
           ,col('hkbn_TOTAL_BYTE_COUNT').alias('HKBN_USAGE')\
           ,col('house_moving_TOTAL_BYTE_COUNT').alias('HOUSE_MOVING_USAGE')\
           ,col('house_renovating_companys_TOTAL_BYTE_COUNT').alias('HOUSE_RENOVATING_COMPANYS_USAGE')\
           ,col('netvigator_TOTAL_BYTE_COUNT').alias('NETVIGATOR_USAGE')\
           ,col('village_TOTAL_BYTE_COUNT').alias('VILLAGE_PROPERTY_AGENT_USAGE')\
           ,col('LANG').alias('LANGUAGE')\
           ).toPandas().to_csv(output_path, index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/app/sna_bdmrdev02/data_out/to_Cognos_De_SMS/De_5G_BB_20230117.csv'

In [ ]:
spark.stop()
conn.close()
sc.stop()